In [1]:
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import glob
import time
import pandas as pd
plt.rcParams["figure.figsize"] = (3,3)

print (time.ctime())

Tue Jul  7 09:01:23 2020


In [2]:
# read a data file, store all data in arrays

file = "fourions_trigger100kHz_300s_W0028_H11-200211-192509-1_cent.csv"

print (file, time.ctime())
data = np.loadtxt(file, dtype= float, delimiter=",", usecols = (0,1,2,3,4,5,6,7,8,9,10,11))


#print data
tid = data[:, 0]    #
tt  = data[:, 1]    #trigger time
y   = data[:, 2]    #y-coordinate
x   = data[:, 3]    #x-coordinate
t   = data[:, 4]    #time of arrival
a   = data[:, 5]    #centroided data (flux of brightest pixel)
A   = data[:, 6]    #centroided data (flux of cluster)
tof = data[:, 7]    #time of flight
n   = data[:, 8]    #pixels in cluster (centroided data)
yc  = data[:, 9]    #corrected y
xc  = data[:, 10]   #corrected x
tof_corr = data[:, 11]*1000     #corrected time of flight

fileCorrection = "fourions_trigger100kHz_300s_W0028_H11-200211-192509-1_LTcorr.csv"
correction = np.loadtxt(fileCorrection, unpack=True, delimiter=",")
TOT_i  = [int(tot/25) for tot in a]
t_corr = t/4096*25 + correction[1][TOT_i]*1000  ### WAS CHANGED FROM ORIGINAL
tof_corr = tof/4096*25 #+ correction[1][TOT_i]*1000   ### WAS CHANGED FROM ORIGINAL
tt_corr = tt/4096*25                                 ###TT TIME CONVERSION TO (ns)

fourions_trigger100kHz_300s_W0028_H11-200211-192509-1_cent.csv Tue Jul  7 09:01:23 2020


In [3]:
data_table = pd.read_csv(file)                                                                       #Create Pandas Table
data_table.rename(columns = {'#TrigId':'tid', '#TrigTime':'tt', '#Col':'y', '#Row':'x', '#ToA':'time', '#ToT[arb]':'center flux', '#ToTtotal[arb]':'cluster flux'}, inplace=True)                    #Rename Columns
data_table.rename(columns = {'#Trig-ToA[arb]':'tof', '#Centroid':'cluster size', '#cent_X':'yc', '#cent_Y':'xc', '#cTrig-ToA[us]':'tof_corr'}, inplace=True)
data_table.drop(columns = {'Unnamed: 12', 'tid', 'tof_corr'}, inplace=True)   #Remove unwanted columns
data_table['tof'] = tof_corr                                   ####SWAP OUT CORRECTED VALUES FOR TOF
data_table['tot-i'] = TOT_i
data_table['tfold'] = (data_table['tof'] + 30)%54.789717
real_time = ((t_corr - t_corr.min()))                          #SWAP OUT CORRECTED VALUES AND START TOA AT ZERO.
data_table['time'] = real_time   
real_tt = ((tt_corr - t_corr.min()))                            #tt translation to match zeroing of TOA
data_table['tt'] = real_tt
data_table

,tt,y,x,time,center flux,cluster flux,tof,cluster size,yc,xc,tot-i,tfold
0,-1.300031e+03,21,41,0.000000e+00,600,1800,1312.762451,5,21.152778,41.138889,24,27.809243
1,5.951586e+04,103,118,6.552211e+04,4450,10000,6006.250000,9,103.405000,118.615000,178,9.381130
2,1.608765e+05,104,121,1.659143e+05,4425,10975,5037.762451,10,104.536446,121.265376,177,27.108487
3,3.129177e+05,174,244,3.180703e+05,2450,6000,5154.431152,6,174.841667,244.320833,98,34.197754
4,5.156382e+05,101,119,5.218867e+05,1675,3900,6251.043701,5,101.282051,119.096154,67,35.015963
...,...,...,...,...,...,...,...,...,...,...,...,...
2571523,3.037412e+11,212,91,3.037412e+11,1400,3725,899.743652,5,212.755034,91.969799,56,53.108180
2571524,3.037413e+11,105,108,3.037413e+11,850,4325,9365.887451,8,105.765896,108.078035,34,26.845844
2571525,3.037415e+11,96,185,3.037415e+11,2950,7275,9525.524902,7,96.142612,185.420962,118,22.114144
2571526,3.037415e+11,164,48,3.037415e+11,2475,4200,6784.118652,6,164.619048,48.583333,99,20.193744


In [4]:
data_table.to_csv('Trigger Data Pandas')

In [5]:
data_table['tof'].max()

10136.981201171875

In [6]:
m = (data_table
     .query("tof<0")
    )
m

,tt,y,x,time,center flux,cluster flux,tof,cluster size,yc,xc,tot-i,tfold
